# Experiment:

Explore the dataset and build a model that can accept the data

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import argparse
import os

In [3]:
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image
#from torchnet.meter import AverageValueMeter
import torch.backends.cudnn as cudnn

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/MyDrive/')
%pwd

Mounted at /content/gdrive


'/content/gdrive/MyDrive'

In [5]:
from utils import *
from model import *

In [6]:
import argparse
import torch

# Default parameters
default_params = {
    'data_dir': './data/',
    'nb_epoch': 50,
    'test_size': 0.1,
    'learning_rate': 0.0001,
    'samples_per_epoch': 64,
    'batch_size': 36,
    'cuda': True,
    'seed': 7
}

# Parse command-line arguments
args = argparse.Namespace(**default_params)

# Adjust CUDA setting based on availability
args.cuda = args.cuda and torch.cuda.is_available()

# Set random seed for reproducibility
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [7]:
def load_data(args):
    """
    Load training data and split it into training and validation set
    """
    #reads CSV file into a single dataframe variable
    data_df = pd.read_csv(os.path.join(os.getcwd(), args.data_dir, 'driving_log.csv'), names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])

    #yay dataframes, we can select rows and columns by their names
    #we'll store the camera images as our input data
    X = data_df[['center', 'left', 'right']].values
    #and our steering commands as our output data
    y = data_df['steering'].values

    #now we can split the data into a training (80), testing(20), and validation set
    #thanks scikit learn
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=args.test_size, random_state=0, shuffle=True)

    return X_train, X_valid, y_train, y_valid

In [8]:
X_train, X_valid, y_train, y_valid = load_data(args)

In [9]:
transformations = transforms.Compose([transforms.Lambda(lambda x: x/127.5 - 1)
                                     ])

In [23]:
X_train

array([['IMG/center_2016_12_01_13_37_51_293.jpg',
        ' IMG/left_2016_12_01_13_37_51_293.jpg',
        ' IMG/right_2016_12_01_13_37_51_293.jpg'],
       ['IMG/center_2016_12_01_13_37_35_908.jpg',
        ' IMG/left_2016_12_01_13_37_35_908.jpg',
        ' IMG/right_2016_12_01_13_37_35_908.jpg'],
       ['IMG/center_2016_12_01_13_37_34_189.jpg',
        ' IMG/left_2016_12_01_13_37_34_189.jpg',
        ' IMG/right_2016_12_01_13_37_34_189.jpg'],
       ...,
       ['IMG/center_2016_12_01_13_35_25_756.jpg',
        ' IMG/left_2016_12_01_13_35_25_756.jpg',
        ' IMG/right_2016_12_01_13_35_25_756.jpg'],
       ['IMG/center_2016_12_01_13_37_02_415.jpg',
        ' IMG/left_2016_12_01_13_37_02_415.jpg',
        ' IMG/right_2016_12_01_13_37_02_415.jpg'],
       ['IMG/center_2016_12_01_13_37_15_051.jpg',
        ' IMG/left_2016_12_01_13_37_15_051.jpg',
        ' IMG/right_2016_12_01_13_37_15_051.jpg']], dtype=object)

In [10]:
#train_set = CarDataset(X_train, y_train, args.data_dir, False,transformations)
train_set = CarDataset3Img(X_train, y_train, args.data_dir,transformations)

In [11]:
valid_set = CarDataset(X_valid, y_valid, args.data_dir, False, transformations)

In [12]:
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [13]:
valid_loader = DataLoader(valid_set, batch_size=args.batch_size, shuffle=False, num_workers=4)

# Build Model

 ### NVIDIA Model
    Image normalization to avoid saturation and make gradients work better.
    Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Drop out (0.5)
    Fully connected: neurons: 100, activation: ELU
    Fully connected: neurons: 50, activation: ELU
    Fully connected: neurons: 10, activation: ELU
    Fully connected: neurons: 1 (output)
    
    the convolution layers are meant to handle feature engineering
    the fully connected layer for predicting the steering angle.
    dropout avoids overfitting
    ELU(Exponential linear unit) function takes care of the Vanishing gradient problem.
    
### Simple Model

```
  (module): CarSimpleModel (
    (conv_layers): Sequential (
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): ELU (alpha=1.0)
      (2): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (3): MaxPool2d (size=(4, 4), stride=(4, 4), dilation=(1, 1))
      (4): Dropout (p = 0.25)
    )
    (linear_layers): Sequential (
      (0): Linear (3648 -> 50)
      (1): ELU (alpha=1.0)
      (2): Linear (50 -> 10)
      (3): Linear (10 -> 1)
    )
  )
```

# Define Train

In [14]:
def toVariable(data, use_cuda):
    input, target = data
    input, target = Variable(input.float()), Variable(target.float())
    if use_cuda:
        input, target = input.cuda(), target.cuda()

    return input, target

In [15]:
# Training
def train(epoch, net, dataloader, optimizer, criterion, use_cuda):
    net.train()
    train_loss = 0

    for batch_idx, (centers, lefts, rights) in enumerate(dataloader):

        optimizer.zero_grad()
        centers, lefts, rights = toVariable(centers, use_cuda), \
                                 toVariable(lefts, use_cuda), \
                                 toVariable(rights, use_cuda)
        datas = [lefts, rights, centers]
        for data in datas:
            imgs, targets = data
            outputs = net(imgs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.data[0]

        if batch_idx % 100 == 0:
            print('Loss: %.3f '
                % (train_loss/((batch_idx+1)*3)))

## Define Test

In [16]:
def valid(epoch, net, validloader, criterion, use_cuda):
    global best_loss
    net.eval()
    valid_loss = 0
    for batch_idx, (inputs, targets) in enumerate(validloader):
        inputs, targets = Variable(inputs.float()), Variable(targets.float())
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        valid_loss += loss.data[0]

        avg_valid_loss = valid_loss/(batch_idx+1)
        if batch_idx % 100 == 0:
            print('Valid Loss: %.3f '
                % (valid_loss/(batch_idx+1)))
        if avg_valid_loss <= best_loss:
            best_loss = avg_valid_loss
            print('Best epoch: ' + str(epoch))
            state = {
                'net': net.module if args.cuda else net,
            }
            torch.save(state, './model3.h5')

In [17]:
net = CarSimpleModel()
optimizer = optim.Adam(net.parameters(), lr=args.learning_rate)

if args.cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

criterion = nn.MSELoss()

/content/model.py:153: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight, mean=0, std=0.02)


In [18]:
best_loss = 0.999

In [19]:
for epoch in range(0,15):
    #optimizer = lr_scheduler(optimizer, epoch, lr_decay_epoch=args.lr_decay_epoch)
    print('\nEpoch: %d' % epoch)
    train(epoch, net, train_loader, optimizer, criterion, args.cuda)
    valid(epoch, net, valid_loader, criterion, args.cuda)


Epoch: 0


FileNotFoundError: ignored

## Save Model

In [ ]:
state = {
        'net': net.module if args.cuda else net,
        }

In [ ]:
torch.save(state, './model4.h5')